In [9]:
## Load the pickle file 

from tensorflow.keras.models import load_model
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle

In [11]:
## Load the model 
model = load_model('model.h5')

## Load encoder and scaler 
with open ('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)
with open ('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
with open ('one_hot_encoder.pkl', 'rb') as file:
    onehot_encoder = pickle.load(file)



In [12]:
## Example input data 
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 50000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

## How to apply one hot encoding 
--> One Hot encoding will be required where we have multiple disfferent categories. Here, we have Geography. 
First transform geography tag using pickle file and then convert it into dataframe



In [13]:
encoded_geo = onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_df = pd.DataFrame(encoded_geo, columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_df

f:\Projects\DL_projects\ANN_project\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [14]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,5,50000,2,1,1,60000


In [15]:
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,5,50000,2,1,1,60000


In [16]:
input_df.drop('Geography', axis=1, inplace=True)
final_df = pd.concat([input_df, geo_df], axis=1)
final_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,5,50000,2,1,1,60000,1.0,0.0,0.0


In [17]:
## Scaling the data 
final_scaled = scaler.transform(final_df)
final_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.00134472, -0.41792108,
         0.80843615,  0.64920267,  0.97481699, -0.70296551,  1.00150113,
        -0.57946723, -0.57638802]])

In [18]:
prediction = model.predict(final_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step


array([[0.02249134]], dtype=float32)

In [19]:
if prediction[0][0] > 0.5:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.
